# LIBS

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Input,Dropout
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import os
import numpy as np
import csv
import matplotlib.pyplot as plt

from utilities import rotate_data, squeeze_data, sequential_arm_rotation
from utilities import VIDEO_LENGTH

# Data

Parameters

In [ ]:
DATA_PATH = os.path.join('model/data')
VIDEO_LENGTH # 25 frames per sequence   
MAX_ANGLE = 10 # degrees
MAX_SQUEEZE = 0.15 # 15% of the image size
MAX_ARM_ROTATION = 4
ARM_ROTATION_PROB = 3/10

Load Data

In [ ]:
with open("model/custom_model_label.csv", encoding='utf-8-sig') as f:
            label_map = csv.reader(f)
            label_map = {
                row[0]: index for index, row in enumerate(label_map)
            }
            actions = list(label_map.keys())

sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(VIDEO_LENGTH):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

X = np.array(sequences)
y = to_categorical(labels).astype(int)

Split Data

In [ ]:
# Step 1: separation 20% of data in val + test
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.20, random_state=43)

# Step 2: from that 20%, use 25% as test → 25% * 20% = 5%
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5, random_state=43)

print(f"X shape: {X.shape}, y shape: {y.shape}")
print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape}, y_test: {y_test.shape}")
print(f"X_val: {X_val.shape}, y_val: {y_val.shape}")

Augmentate Data

In [ ]:
n_example = X_train.shape[0]
rot_angles = np.random.uniform(low=-MAX_ANGLE, high=MAX_ANGLE, size=n_example)
rotated  = rotate_data(X_train, rot_angles)
squeezed = squeeze_data(rotated, MAX_SQUEEZE)
arm_rotated = sequential_arm_rotation(squeezed, MAX_ARM_ROTATION, ARM_ROTATION_PROB)

print(f"X_rotated: {rotated.shape}")
print(f"X_squeezed: {squeezed.shape}")
print(f"X_arm_rotated: {arm_rotated.shape}")

AX_train = np.concatenate((X_train, arm_rotated), axis=0)
Ay_train = np.concatenate((y_train, y_train), axis=0)

print(f"AX_train shape: {AX_train.shape}, AY_train shape: {Ay_train.shape}")

Logs

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
np.random.randint(1, 2 + 1)



# MODEL

Def Model

In [ ]:
model = Sequential()
model.add(Input(shape=(X.shape[1:])))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2)) 
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(actions), activation='softmax'))

Compile Model

In [ ]:
optimizer = Adam(learning_rate=0.0001)
early_stop = EarlyStopping(
    monitor='val_loss',        # o 'val_categorical_accuracy'
    patience=8,                # numero di epoche senza miglioramento prima di fermarsi
    restore_best_weights=True # ripristina i pesi migliori
)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Fit Model (Only When Re-Run the Model)

In [ ]:
history = model.fit(AX_train, Ay_train, epochs=500, batch_size=50, validation_data=(X_val, y_val), callbacks=[early_stop])

Trainig graph

In [ ]:
epochs = range(1, len(history.history['categorical_accuracy']) + 1)
plt.plot(epochs,history.history['categorical_accuracy'], label='Training Accuracy')
plt.plot(epochs,history.history['val_categorical_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

plt.plot(epochs,history.history['loss'], label='Training Loss')
plt.plot(epochs,history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.title('Loss')
plt.show()


val_losses = history.history['val_loss']
best_epoch = np.argmin(val_losses)
print(f"The best weights were obtained at the epoch {best_epoch + 1} with val_loss = {val_losses[best_epoch]:.4f}")

Load Model

In [ ]:
#model = load_model('model/save_model.keras')

Save Model

In [ ]:
model.save('model/save_model.keras')

Sumary Model

In [ ]:
model.summary()

# Evaluation

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,classification_report

Test data

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

Confusion Matrix

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

Accuracyt Score (1 is good, 0 bad)

In [ ]:
accuracy_score(ytrue, yhat)
print(f"Accuracy: {accuracy_score(ytrue, yhat):.4f}")
print("\nClassification Report (includes Precision, Recall, F1 per class):")
report = classification_report(ytrue, yhat)
print(label_map)
print(report)